# CHARISMA 
## Demo calibration notebook
Uses https://github.com/h2020charisma/ramanchada2 library | [Documentation](https://h2020charisma.github.io/ramanchada2/ramanchada2.html) | [Installation](https://pypi.org/project/ramanchada2/)

Steps:
- Read the metadata template
    - Metadata entry template (Excel) available at [Template Wizard](https://enanomapper.adma.ai/projects/enanomapper/datatemplates/pchem/index.html?template=CHARISMA_RR) [1]
- Read the Neon and Si spectra listed in the metadata template
    - supported data formats (spc, .sp, .spa, .wdf, .ngs, .jdx, .dx, .txt, .csv, .rruf)
- Derive calibration model
    - Derive calibration curve using Neon spectra and reference Neon peaks
    - Laser zeroing using Si spectra and reference peak
    - saves the calibraiton model
- Apply calibration model
    - Reads Polystyrene spectra using metadata template    
    - Applies the calibration model 

***
[1] Jeliazkova, Nina, et al. 2024. “A Template Wizard for the Cocreation of Machine-Readable Data-Reporting to Harmonize the Evaluation of (Nano)Materials.” Nature Protocols, May. https://doi.org/10.1038/s41596-024-00993-1.

In [31]:
# imports
import os.path
import pandas as pd
import ramanchada2 as rc2
from pathlib import Path
import shutil
import matplotlib.pyplot as plt
import logging

# set folder
config_root = "."

def read_template(file):
    _path = os.path.join(config_root,file)
    df = pd.read_excel(_path, sheet_name='Files')
    df.columns = ['sample', 'measurement', 'filename', 'optical_path', 'laser_power_mw', 
              'humidity', 'temperature', 'date', 'time']
    df_meta = pd.read_excel(_path, sheet_name='Front sheet', skiprows=4)
    df_meta.columns = ['optical_path', 'instrument_make', 'instrument_model', 'wavelength','collection_optics','slit_size','grating','pin_hole_size','collection_fibre_diameter','notes']    
    df_merged = pd.merge(df, df_meta, on='optical_path', how='left')

    # Open the Excel file and read specific cells directly
    with pd.ExcelFile(_path) as xls:
        provider = xls.parse('Front sheet', usecols="B", nrows=1, header=None).iloc[0, 0]
        investigation = xls.parse('Front sheet', usecols="F", nrows=1, header=None).iloc[0, 0]
    df_merged["provider"] = provider
    df_merged["investigation"] = investigation
    df_merged["source"] = file
    return df_merged


import warnings
warnings.filterwarnings("ignore")
logging.getLogger("spc_io").setLevel(logging.ERROR)

In [ ]:
#template_file = "Template Raman Reporting_LBF_532.xlsx"
template_file = "Template Raman Reporting_ZLM785.xlsx"
metadata = read_template(template_file)    
metadata[["investigation","sample","filename","wavelength","optical_path","instrument_make","instrument_model","provider"]].style.set_properties(subset=['filename'], **{'text-align': 'right'}).hide(axis='index')

In [ ]:
neon_tag="Neon"
si_tag = "S0B"
pst_tag="PST"

In [ ]:
# Load Ne and Si spectra
## Load Ne spectrum
meta_neon = metadata.loc[metadata["sample"]==neon_tag]
_file = os.path.join(config_root,meta_neon.iloc[0]["filename"])
spe_neon = rc2.spectrum.from_local_file(_file)

## Load Si spectrum
meta_si = metadata.loc[metadata["sample"]==si_tag]
_file = os.path.join(config_root,meta_si.iloc[0]["filename"])
spe_si = rc2.spectrum.from_local_file(_file)

# Plot
fig, ax = plt.subplots(1, 2, figsize=(15,2))   
spe_neon.plot(label=neon_tag,color="red",ax=ax[0])
ax[0].set_xlabel("cm-1")
spe_si.plot(label=si_tag,color="blue",ax=ax[1])
ax[1].set_xlabel("cm-1")

In [ ]:
# clean .cha files from previous run
for tag in [neon_tag,si_tag, pst_tag]:
    cha_file = os.path.join(config_root,"{}.cha".format(tag))
    if os.path.exists(cha_file):
        os.remove(cha_file)


In [ ]:
# write into .cha file
cha_file = os.path.join(config_root,"{}.cha".format(neon_tag))
spe_neon.write_cha(cha_file,dataset = "/raw")
print(os.path.basename(cha_file))

cha_file = os.path.join(config_root,"{}.cha".format(si_tag))
spe_si.write_cha(os.path.join(config_root,"{}.cha".format(si_tag)),dataset = "/raw")
print(os.path.basename(cha_file))

In [ ]:
# Process spectra
# crop
spe_si = spe_si.trim_axes(method='x-axis',boundaries=(520.45-200,520.45+200))
spe_neon = spe_neon.trim_axes(method='x-axis',boundaries=(100,max(spe_neon.x)))

## baseline  SNIP
kwargs = {"niter" : 40 }
spe_neon = spe_neon.subtract_baseline_rc1_snip(**kwargs)  
spe_si = spe_si.subtract_baseline_rc1_snip(**kwargs)  

### write as dataset in .cha file
spe_neon.write_cha(os.path.join(config_root,"{}.cha".format(neon_tag)),dataset = "/baseline")
spe_si.write_cha(os.path.join(config_root,"{}.cha".format(si_tag)),dataset = "/baseline")

## normalize min/max
spe_neon = spe_neon.normalize()        
spe_si = spe_si.normalize()        

### write as dataset in .cha file
spe_neon.write_cha(os.path.join(config_root,"{}.cha".format(neon_tag)),dataset = "/normalized")
spe_si.write_cha(os.path.join(config_root,"{}.cha".format(si_tag)),dataset = "/normalized")

# Plot
fig, ax = plt.subplots(1, 2, figsize=(15,2))   
ax[0].set_xlabel("cm-1")
ax[1].set_xlabel("cm-1")
spe_neon.plot(label=neon_tag,color="red",ax=ax[0])
spe_si.plot(label=si_tag,color="blue",ax=ax[1])

plt.show()


In [ ]:
# wavelength
laser_wl = metadata["wavelength"].unique()[0]
laser_wl

In [ ]:
import ramanchada2.misc.constants  as rc2const
noise_factor = 1.5
neon_wl = {
    785: rc2const.neon_wl_785_nist_dict,
    633: rc2const.neon_wl_633_nist_dict,
    532: rc2const.neon_wl_532_nist_dict
}

## Derive the calibration model

In [ ]:
from ramanchada2.protocols.calibration import CalibrationModel

def calibration_model(laser_wl,spe_neon,spe_sil):
    calmodel = CalibrationModel(laser_wl)
    calmodel.prominence_coeff = 3
    model_neon = calmodel.derive_model_curve(spe_neon,neon_wl[laser_wl],spe_units="cm-1",ref_units="nm",find_kw={},fit_peaks_kw={},should_fit = False,name="Neon calibration")
    spe_sil_ne_calib = model_neon.process(spe_sil,spe_units="cm-1",convert_back=False)
    find_kw = {"prominence" :spe_sil_ne_calib.y_noise * calmodel.prominence_coeff , "wlen" : 200, "width" :  1 }
    model_si = calmodel.derive_model_zero(spe_sil_ne_calib,ref={520.45:1},spe_units="nm",ref_units="cm-1",find_kw=find_kw,fit_peaks_kw={},should_fit=True,name="Si calibration")
    model_si.peaks.to_csv(os.path.join(config_root,template_file.replace(".xlsx","peaks.csv")),index=False)

    spe_sil_calib = model_si.process(spe_sil_ne_calib,spe_units="nm",convert_back=False)

    fig, ax =plt.subplots(1,1,figsize=(12,2))
    spe_sil.plot(label="{} original".format(si_tag),ax=ax)
    spe_sil_calib.plot(ax = ax,label="{} laser zeroed".format(si_tag),fmt=":")
    ax.set_xlim(520.45-50,520.45+50)    
    ax.set_xlabel("cm-1")
    
    return calmodel

def apply_calibration_x(calmodel: CalibrationModel, old_spe: rc2.spectrum.Spectrum, spe_units="cm-1"):
    new_spe = old_spe
    model_units = spe_units
    for model in calmodel.components:
        if model.enabled:
            new_spe = model.process(new_spe, model_units, convert_back=False)
            model_units = model.model_units
    return new_spe

In [ ]:
calmodel = calibration_model(laser_wl,spe_neon,spe_si)

In [ ]:
# save the calibration model
_calfile = os.path.join(config_root,template_file.replace(".xlsx",".calibration"))
print(os.path.basename(_calfile))
calmodel.save(_calfile) 
# we can load this file later e.g.
# calmodel = CalibrationModel.from_file(_calfile)   
# and apply to a new spectrum e.g.
# spe_pst_calibrated = calmodel.apply_calibration_x(spe_pst)  

## Apply the calibration model

In [ ]:
# Apply the calibration model
from ramanchada2.spectrum import from_chada


## Load PST spectrum
meta_pst = metadata.loc[metadata["sample"]==pst_tag]
_file = os.path.join(config_root,meta_pst.iloc[0]["filename"])

spe_pst = rc2.spectrum.from_local_file(_file)

spe_pst = spe_pst.trim_axes(method='x-axis',boundaries=(100,max(spe_pst.x)))

# Plot
fig, ax = plt.subplots(2, 1, figsize=(15,4))   
ax[1].set_xlabel("cm-1")
ax[0].set_title(pst_tag)
spe_pst.plot(ax=ax[0],label="raw")

#Preprocess
kwargs = {"niter" : 40 }
spe_pst = spe_pst.subtract_baseline_rc1_snip(**kwargs) 
spe_pst.plot(ax=ax[0],label="baseline removed")
spe_pst = spe_pst.normalize()
spe_pst.plot(ax=ax[1],label="normalized")


# calibrate
spe_pst_calibrated = calmodel.apply_calibration_x(spe_pst) 
spe_pst_calibrated.plot(ax=ax[1],label="calibrated",color="red")
plt.show()

In [ ]:
spe_pst_calibrated.write_cha(os.path.join(config_root,"{}.cha".format(si_tag)),dataset = "/calibrated")

In [ ]:
# export calibrated spectrum into text format
pd.DataFrame(zip(spe_pst_calibrated.x,spe_pst_calibrated.y), columns=['X', 'Y'])

> This project has received funding from the European Union's Horizon 2020 research and innovation programme under grant agreement 952921 CHARISMA